In [1]:
import pandas as pd
import numpy as np

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NFLX']
folder = "/Users/pavelmanyakin/Desktop/Data/" #This should be changed to your folder with data
window_size = 60
Npi = 100
bandwidth = 0.1
all_prices = []
for ticker in tickers:
    file_path = f"{folder}{ticker}.csv"
    df = pd.read_csv(file_path, parse_dates=['Date'])
    price_col = 'Adjusted Close' if 'Adjusted Close' in df.columns else 'Adj Close'
    df = df[['Date', price_col]].rename(columns={price_col: ticker})
    all_prices.append(df)

data = all_prices[0]
for df in all_prices[1:]:
    data = data.merge(df, on='Date', how='inner')
data = data.set_index('Date')
data = data.loc["2012-01-01":"2021-12-31"]
log_returns = np.log(data / data.shift(1)).dropna()

def create_sequences(log_returns, window):
    sequences = []
    for i in range(len(log_returns) - window):
        seq = log_returns.iloc[i:i+window].values
        sequences.append(seq)
    return np.array(sequences)

data_series = create_sequences(log_returns, window_size)

def quartic_kernel(x, h):
    norm = np.linalg.norm(x) / h
    return ((1 - norm**2)**2) / h if norm < 1 else 0.0

def Fi(t, xti, x, xtip1, ti, tip1):
    dt1 = tip1 - t
    dt2 = tip1 - ti
    term1 = -np.linalg.norm(xtip1 - x)**2 / (2 * dt1)
    term2 = np.linalg.norm(xtip1 - xti)**2 / (2 * dt2)
    return np.exp(term1 + term2)

def estimate_drift(t, x, xi, data_series, ti, tip1, h):
    num = np.zeros_like(x)
    denom = 0.0
    for sample in data_series:
        if tip1 >= sample.shape[0]:
            continue
        xti = sample[ti]
        xtip1 = sample[tip1]
        Fi_val = Fi(t, xti, x, xtip1, ti, tip1)
        weight = Fi_val
        for j in range(ti):
            weight *= quartic_kernel(xi[j] - sample[j], h)
        num += weight * (xtip1 - x)
        denom += weight
    return (1 / (tip1 - t)) * num / (denom + 1e-8)

def sbts_simulation(data_series, Npi=100, h=0.1):
    M, N, d = data_series.shape
    x = [np.zeros(d)]
    for i in range(N - 1):
        y = x[i].copy()
        for k in range(Npi - 1):
            tpi_k = i + k / Npi
            drift = estimate_drift(tpi_k, y, np.array(x), data_series, ti=i, tip1=i+1, h=h)
            eps = np.random.randn(d)
            y = y + (1 / Npi) * drift + (1 / np.sqrt(Npi)) * eps
        x.append(y)
    return np.array(x)

simulated = sbts_simulation(data_series, Npi=Npi, h=bandwidth)
df_sim = pd.DataFrame(simulated, columns=tickers)
print(df_sim)

/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_65921/3850030659.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_65921/3850030659.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_65921/3850030659.py:15: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(file_path, parse_dates=['Date'])
/var/folders/ml/w52lgnpd6

        AAPL      MSFT      GOOG       AMZN      NFLX
0   0.000000  0.000000  0.000000   0.000000  0.000000
1   0.138859  0.145743 -0.084411   0.154681 -0.011797
2  -0.073217  0.010271  0.033923   0.126346  0.023622
3  -0.330390 -1.238127 -0.037263  -0.473143  0.685930
4   0.054107 -1.577321  0.332130  -0.096049  0.639518
5   1.687088 -1.971313 -0.049457   1.978164  0.580290
6   1.755869 -0.830319 -0.476801   2.792832  0.418187
7   1.562430 -1.408371  0.508415   2.237805 -0.340348
8   0.981890 -1.267140 -1.132726   1.442785 -0.769957
9   0.808574  0.983540  0.794294   0.967555 -0.525996
10 -0.464085  0.388263  0.828082   1.960611  0.692028
11  0.312592  0.973832  1.824963   0.678068  0.530684
12  0.503126  0.217847  1.189819   2.382189  2.203007
13 -0.126662 -2.038947  2.321617   3.196147  2.759763
14 -0.971091 -2.635684  3.297516   3.655938  3.732111
15  0.623695 -2.780607  2.054232   3.054466  2.107440
16  1.297298 -2.855359  2.191788   3.994580  0.884455
17  2.080453 -3.728527  3.07

In [5]:
returns = df_sim.diff().dropna()

mu = returns.mean().values
cov = returns.cov().values
inv_cov = np.linalg.inv(cov)
ones = np.ones(len(mu))

w_sharpe = inv_cov @ mu
w_sharpe /= np.sum(w_sharpe)

portfolios = pd.DataFrame([w_sharpe],
                          columns=df_sim.columns,
                          index=['MaxSharpe'])
print(portfolios)

              AAPL      MSFT      GOOG      AMZN      NFLX
MaxSharpe  0.46623 -0.372682  0.358047  0.787741 -0.239337


In [6]:
import os

tickers = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'NFLX']
folder = "/Users/pavelmanyakin/Desktop/Data/" #This should be changed to your folder with data
start_date = '2022-01-01'
end_date = '2022-03-01'
initial_investment = 100000
weights = np.array([0.46623, -0.372682, 0.358047, 0.787741, -0.239337])

close_prices = pd.DataFrame()

for ticker in tickers:
    filepath = os.path.join(folder, f"{ticker}.csv")
    df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
    close_prices[ticker] = df['Adjusted Close']

close_prices = close_prices.loc[start_date:end_date]

start_prices = close_prices.iloc[0]
end_prices = close_prices.iloc[-1]

shares_bought = (initial_investment * weights) / start_prices

final_value = np.sum(shares_bought * end_prices)

print(f"Initial investment: ${initial_investment:,.2f}")
print(f"Final portfolio value after 2 months: ${final_value:,.2f}")
print(f"Return: {(final_value / initial_investment - 1) * 100:.2f}%")


Initial investment: $100,000.00
Final portfolio value after 2 months: $64,986.37
Return: -35.01%


/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_65921/2369887183.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_65921/2369887183.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
/var/folders/ml/w52lgnpd681dr_dyz7dhqz9r0000gn/T/ipykernel_65921/2369887183.py:14: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  df = pd.read_csv(filepath, parse_dates=['